<a href="https://colab.research.google.com/github/pastoril10/Sinais_de_trading_e_backtesting-Bitcoin/blob/main/Cria%C3%A7%C3%A3o_de_sinais_de_trading_e_backtesting_Bitcoin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instalação e importação das bibliotecas

Instalando as bibliotacas para nos dar acesso aos dados direto da binance e a biblioteca de visualização plotly

In [37]:
!pip instal plotly
!pip install yfinance

ERROR: unknown command "instal" - maybe you meant "install"


Importando as bibliotecas que iremos utilizar ao longo do nosso projeto

In [38]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline  as py
import plotly.express as px
import yfinance as yf

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#2. Importação dos dados e primeiras adequações

In [39]:
#Data de inicio e término na coleta dos dados
start = "2020-01-01"
end = "2022-05-10"

In [40]:
#Importando os dados direto do yahoo finance
btc_usd = yf.download("BTC-USD", start = start, 
                 end = end)

[*********************100%***********************]  1 of 1 completed


In [41]:
#Visualizando as 10 primeiras linhas do nosso DF
btc_usd.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,7200.174316,18565664997
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,6985.470215,20802083465
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,7344.884277,28111481032
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,7410.656738,18444271275
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,7411.317383,19725074095


In [42]:
#Verificando o tipo de dados que nós temos
type(btc_usd)

pandas.core.frame.DataFrame

In [43]:
# Dropar todas as colunas da 4 pra frente
btc_usd = btc_usd.drop(btc_usd.columns[4:], axis = 1)

In [44]:
btc_usd.head()

,Open,High,Low,Close
Date,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383


In [45]:
#verificando o tipo de cada coluna
btc_usd.dtypes

Open     float64
High     float64
Low      float64
Close    float64
dtype: object

#3. Análise Quantitativa

## Cotação da criptomoeda Bitcoin (BTC)

In [46]:
trace1 = go.Scatter(
                    x = btc_usd.index,
                    y = btc_usd["Close"],
                    mode = "lines",
                    #name = "Bitcoin,
                    marker = dict(color = 'blue'))

data = [trace1]
layout = dict(title = 'Acompanhe a cotação da criptomoeda Bitcoin(BTC)',
              xaxis= dict(title= 'Periodo (em dias)', ticklen= 5,zeroline= False),
              yaxis= dict(title= "<b> Preço do Bitcion (R$)"))

fig = dict(data = data, layout = layout)
py.iplot(fig)

## Retorno diario

In [47]:
#Retorno diario (O valor de Close de hoje - o valor de Close de ontem)
btc_usd["retorno"] = btc_usd.Close.pct_change()

In [48]:
#Gerando subplots
fig = make_subplots(rows=2, cols=1)

fig.append_trace(go.Scatter(x = btc_usd.index, y = btc_usd["retorno"], name = "Dispersão", mode = 'lines'), row=1, col=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray',showline=True, linewidth=1, linecolor='gray', row = 1, col = 1)
fig.update_yaxes(title_text = "Preço em Reais R$", showgrid=True, gridwidth=1, gridcolor='lightgray',showline=True, linewidth=1, linecolor='gray', row = 1, col = 1)

#Histograma
fig.append_trace(go.Histogram(x = btc_usd.retorno, nbinsx= 300, name = "Histograma" ), row=2, col=1)
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray',showline=True, linewidth=1, linecolor='gray', row = 2, col = 1)
fig.update_yaxes(title_text = "Frequency", showgrid=True, gridwidth=1, gridcolor='lightgray',showline=True, linewidth=1, linecolor='gray', row = 2, col = 1)


# o comando fig.update_layout nos permite alterar o layout do gráfico
fig.update_layout(plot_bgcolor = 'white',
font = {'family': 'Arial','size': 16,'color': 'black'},
colorway=["red", "blue"])

fig.update_layout(height=800, width=800, title_text="Retorno diario")
fig.show()

In [49]:
btc_usd.retorno.describe()

count    860.000000
mean       0.002492
std        0.039254
min       -0.371695
25%       -0.015297
50%        0.002284
75%        0.020696
max        0.187465
Name: retorno, dtype: float64

## Volatilidade

In [50]:
# Janela de volatilidade
vol_30 = btc_usd["retorno"].rolling(window = 30).std()

In [51]:
trace1 = go.Scatter(
                    x = vol_30.index,
                    y = vol_30,

                    mode = "lines+markers",
                    #name = "Bitcoin",
                    marker = dict(color = 'red'))

data = [trace1]
layout = dict(title = 'Janela de volatilidade',
              xaxis= dict(title= 'Periodo (em dias)', ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Volatilidade'))

fig = dict(data = data, layout = layout)
py.iplot(fig)

Verificamos que o periodo com a maior volatilidade foi no inicio da pandemia do Corona Virus - do dia 11 de março de 2020 a 19 de abril de 2020

Outros dois periodos de maior volatilidade são observados. 
No inicio de 2021 até meados do mes de abril e de meados de abril até final de Julho. 

## Criando médias móveis

In [52]:
# variação minínima para ações no preço
tick_min = 1 
MM_5 = 5
MM_25 = 25

In [53]:
## Cálculo Média Móvel de 9 períodos
MM_5p = btc_usd['Close'].rolling(window = MM_5).mean()
## Cálculo Média Móvel de 30 períodos
MM_25p = btc_usd['Close'].rolling(window = MM_25).mean()

In [54]:
btc = btc_usd.copy()

In [55]:
btc["MM_5p"] = MM_5p
btc["MM_25p"] = MM_25p

In [56]:
#GRÁFICO MÉDIA MOVEL

#cria figura
fig = make_subplots(rows=1, cols=1)
#Adiciona o gráfico de candlestick com os parametros OHLC no eixo y e no eixo x o index com as datas
fig.add_trace(go.Candlestick(name='Bitcoin_BRL', 
                             x=btc.index, 
                             open=btc['Open'], 
                             high = btc['High'], 
                             low=btc['Low'], 
                             close=btc['Close'],
                             showlegend=True), row=1,col=1)

# criando MM_5p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_5p', x=btc.index, y=btc.MM_5p, marker_color='blue'),row=1,col=1)
# criando MM_25p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_25p', x=btc.index, y=btc.MM_25p, marker_color='black'),row=1,col=1)

#Personalizar o eixo Y
fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)
fig.update_xaxes(title_text="<b> Periodo em dias")

#Personalizar aspectos da figura em geral
fig.update_layout(xaxis_rangeslider_visible=False,
                  width=1000,
                  height=500)
#Mostra a figura
fig.show()

# 4. Criação dos sinais de trading e backtesting

## 4.1. Construção do sinal de COMPRA

Regra para compra
MM_5p cruza a MM_25p para cima - Sinal de compra


In [57]:
#Criação do booleano da Regra 1 deixando true or false
regra1 = (btc["MM_5p"] > btc["MM_25p"]) & (btc["MM_5p"].shift(1) < btc["MM_25p"].shift(1))
btc["regra1"] = regra1
btc

,Open,High,Low,Close,retorno,MM_5p,MM_25p,regra1
Date,,,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,NaN,NaN,NaN,False
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,-0.029819,NaN,NaN,False
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,0.051452,NaN,NaN,False
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,0.008955,NaN,NaN,False
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,0.000089,7270.500586,NaN,False
...,...,...,...,...,...,...,...,...
2022-05-06,36573.183594,36624.359375,35482.132812,36040.921875,-0.014606,37718.842969,39433.204375,False
2022-05-07,36042.503906,36129.925781,34940.824219,35501.953125,-0.014954,37113.367969,39248.195156,False
2022-05-08,35502.941406,35502.941406,33878.964844,34059.265625,-0.040637,36375.130469,38963.896562,False


In [58]:
#Gráfico para mostrar onde a regra 1 = True

fig = make_subplots(rows=1, cols=1)

data_range_btc = pd.date_range(start='2020-03-28',end='2020-04-04')
btc_index_range = btc[btc.index.isin(data_range_btc)]
btc_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', 
                             x=btc_index_range.index, 
                             open=btc_index_range['Open'], 
                             high = btc_index_range['High'], 
                             low=btc_index_range['Low'], 
                             close=btc_index_range['Close'],
                             showlegend=True), row=1,col=1)

# criando MM_5p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_5p', x=btc_index_range.index, y=btc_index_range.MM_5p, marker_color='blue'),row=1,col=1)
# criando MM_25p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_25p', x=btc_index_range.index, y=btc_index_range.MM_25p, marker_color='black'),row=1,col=1)

fig.add_annotation(x = "2020-04-01",
                   y = 6600,
                   text="<b>Sinal de compra",
                   font=dict(family="Arial",size=12,color="black"))

fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False,title_text='<b>Sinal de compra - MM_5p cruza a MM_25p para cima' ,width=1000 ,height=500)
fig.show()

Regra para venda

MM_5p cruza a MM_25p para baixo - Sinal de venda


In [59]:
regra2 = (btc["MM_5p"].shift(1) > btc["MM_25p"].shift(1)) & (btc["MM_5p"] < btc["MM_25p"])
btc["regra2"] = regra2
btc

,Open,High,Low,Close,retorno,MM_5p,MM_25p,regra1,regra2
Date,,,,,,,,,
2020-01-01,7194.892090,7254.330566,7174.944336,7200.174316,NaN,NaN,NaN,False,False
2020-01-02,7202.551270,7212.155273,6935.270020,6985.470215,-0.029819,NaN,NaN,False,False
2020-01-03,6984.428711,7413.715332,6914.996094,7344.884277,0.051452,NaN,NaN,False,False
2020-01-04,7345.375488,7427.385742,7309.514160,7410.656738,0.008955,NaN,NaN,False,False
2020-01-05,7410.451660,7544.497070,7400.535645,7411.317383,0.000089,7270.500586,NaN,False,False
...,...,...,...,...,...,...,...,...,...
2022-05-06,36573.183594,36624.359375,35482.132812,36040.921875,-0.014606,37718.842969,39433.204375,False,False
2022-05-07,36042.503906,36129.925781,34940.824219,35501.953125,-0.014954,37113.367969,39248.195156,False,False
2022-05-08,35502.941406,35502.941406,33878.964844,34059.265625,-0.040637,36375.130469,38963.896562,False,False


In [60]:
#Gráfico para explicar o momento onde a regra 2 = True
fig = make_subplots(rows=1, cols=1)

data_range_btc = pd.date_range(start='2020-05-20',end='2020-05-28')
btc_index_range = btc[btc.index.isin(data_range_btc)]
btc_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_index_range.index, 
                             open=btc_index_range['Open'], 
                             high = btc_index_range['High'], 
                             low=btc_index_range['Low'], 
                             close=btc_index_range['Close'],
                             showlegend=True), row=1,col=1)

# criando MM_5p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_5p', x=btc_index_range.index, y=btc_index_range.MM_5p, marker_color='blue'),row=1,col=1)
# criando MM_25p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_25p', x=btc_index_range.index, y=btc_index_range.MM_25p, marker_color='black'),row=1,col=1)

fig.add_annotation(x = "2020-05-24",
                   y = 9200,
                   text="<b>Sinal de venda",
                   font=dict(family="Arial",size=12,color="black"))

fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=1)

fig.update_layout(xaxis_rangeslider_visible=False,title_text='<b>Sinal de venda - MM_5p cruza a MM_25p para baixo' ,width=1000 ,height=500)
fig.show()

Criando um DF com os sinal de compra e venda



In [61]:
#Criando um df com os dias de compra
sinal_compra = btc[btc['regra1'] == True] 
len(sinal_compra)


19

In [62]:
#Criando um df com os dias de venda
sinal_venda = btc[btc['regra2'] == True] 
len(sinal_venda)

20

Verificamos que o DF de venda é maior que o DF de compra. Isso se deve ao fato de que no intervalo desejado, a MM curta cruza a média móvel longa para baixo - sinal de venda. 
Para ajustar, iremos excluir a primeira linha do DF sinal_venda.

In [63]:
sinal_venda = sinal_venda.iloc[1:]

In [64]:
compra_venda = pd.concat([sinal_compra, sinal_venda], axis = 0).sort_index()
compra_venda

,Open,High,Low,Close,retorno,MM_5p,MM_25p,regra1,regra2
Date,,,,,,,,,
2020-04-01,6437.319336,6612.573730,6202.373535,6606.776367,0.026113,6327.899902,6300.874277,True,False
2020-05-24,9212.283203,9288.404297,8787.250977,8790.368164,-0.045489,9157.395117,9230.676211,False,True
2020-05-30,9438.914062,9704.030273,9366.729492,9700.414062,0.027682,9336.271875,9321.203555,True,False
2020-06-15,9386.035156,9504.860352,8990.175781,9450.702148,0.006809,9423.078516,9480.670156,False,True
2020-07-10,9273.357422,9287.471680,9118.001953,9278.807617,0.000091,9322.572070,9285.999102,True,False
2020-07-14,9238.703125,9283.841797,9171.661133,9243.213867,-0.000043,9256.496484,9257.420547,False,True
2020-07-21,9162.514648,9407.262695,9149.389648,9374.887695,0.022987,9207.073828,9205.610352,True,False
2020-08-25,11773.588867,11778.299805,11189.850586,11366.134766,-0.034690,11615.978516,11678.830703,False,True
2020-09-19,10933.752930,11134.092773,10909.618164,11094.346680,0.013684,10951.955859,10862.360781,True,False


In [83]:
## GRÁFICO QUE DEMONSTRA AS ENTRADAS DA ESTRATÉGIA
fig = make_subplots(rows=1, cols=2)

## Figura 1
data_range_btc = pd.date_range(start='2020-07-15', 
                               end='2020-08-31')

btc_index_range = btc[btc.index.isin(data_range_btc)]


fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_index_range.index, 
                             open=btc_index_range['Open'], 
                             high = btc_index_range['High'], 
                             low=btc_index_range['Low'], 
                             close=btc_index_range['Close'],
                             showlegend=True), row=1,col=1)

fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)",
                 row=1, col=1)

# criando MM_5p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_5p', 
                         x=btc_index_range.index, 
                         y=btc_index_range.MM_5p, 
                         marker_color='blue'),row=1,col=1)
# criando MM_25p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_25p', 
                         x=btc_index_range.index, 
                         y=btc_index_range.MM_25p, 
                         marker_color='black'),row=1,col=1)


fig.add_vrect(x0='2020-07-18 12:00', 
              x1='2020-08-26 12:00', 
              fillcolor="green", 
              opacity=0.25, 
              line_width=0,
              row=1, col=1)

fig.add_trace(go.Scatter(x=['2020-08-05'],
                         y=[10500],
                         mode="text",
                         text="<b>Zoom",
                         textfont_size=16,
                         textposition="top center",
                         showlegend=False,
                         ),
              row=1, col=1)

## Figura 2
data_range_btc = pd.date_range(start='2020-07-18',
                                  end='2020-08-26')

btc_index_range = btc[btc.index.isin(data_range_btc)]
btc_index_range

fig.add_trace(go.Candlestick(name='Bitcoin_BRL', x=btc_index_range.index, 
                             open=btc_index_range['Open'], 
                             high = btc_index_range['High'], 
                             low=btc_index_range['Low'], 
                             close=btc_index_range['Close'],
                             showlegend=False), row=1,col=2)


fig.update_yaxes(title_text="<b> Preço Bitcoin (R$)", row=1, col=2)

fig.add_annotation(x='2020-7-21',
                   y=9375, 
                   xref="x", 
                   yref="y",
                   text="<b>Buy",
                   showarrow=True,
                   font=dict(family="Arial",size=16,color="black"),
                   align="right",arrowhead=2,
                   arrowsize=1,
                   arrowwidth=2,
                   arrowcolor="black",row=1, col=2)


fig.add_annotation(x='2020-8-25',
                   y=11773, 
                   xref="x", 
                   yref="y",
                   text="<b>Sell",
                   showarrow=True,
                   font=dict(family="Arial",size=16,color="black"),
                   align="right",arrowhead=2,
                   arrowsize=1,
                   arrowwidth=2,
                   arrowcolor="black",row=1, col=2)

# criando MM_5p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_5p', 
                         x=btc_index_range.index, 
                         y=btc_index_range.MM_5p, 
                         showlegend=False,
                         marker_color='blue'),row=1,col=2)
# criando MM_25p com o mesmo racional de eixo de cima
fig.add_trace(go.Scatter(name='MM_25p', 
                         x=btc_index_range.index, 
                         y=btc_index_range.MM_25p,
                         showlegend=False,
                         marker_color='black'),row=1,col=2)


## Geral
fig.update_layout(xaxis_rangeslider_visible=False,
                  xaxis2_rangeslider_visible=False,
                  legend=dict(orientation="h",
                              xanchor="right",x=1,
                              yanchor="bottom",y=1.1),
                  title_text='Cruzamento MM Curta e MM longa' ,
                  width=1000 ,height=500)

fig.show()

Contabilidade da operação: Vamos criar um df com os valores de compra e de venda para realizar a contabilidade das operações

In [66]:
preco_compra = sinal_compra["Open"].reset_index().drop(columns = "Date").rename(columns = {"Open":"Preco_compra"})
preco_venda = sinal_venda["Open"].reset_index().drop(columns = "Date").rename(columns = {"Open":"Preco_venda"})

df_resultado = pd.concat([preco_compra, preco_venda], axis = 1)
df_resultado 

,Preco_compra,Preco_venda
0,6437.319336,9212.283203
1,9438.914062,9386.035156
2,9273.357422,9238.703125
3,9162.514648,11773.588867
4,10933.752930,10535.492188
5,10695.575195,10550.440430
6,10927.913086,18051.320312
7,19144.492188,30817.625000
8,35510.820312,46194.015625
9,52272.968750,52726.746094


In [67]:
#Agora vamos subtrair o preço de venda do preço de compra para termos os valores nominais líquidos do resultado do trade
df_resultado['Resultado'] = df_resultado['Preco_venda'] - df_resultado['Preco_compra']
df_resultado

,Preco_compra,Preco_venda,Resultado
0,6437.319336,9212.283203,2774.963867
1,9438.914062,9386.035156,-52.878906
2,9273.357422,9238.703125,-34.654297
3,9162.514648,11773.588867,2611.074219
4,10933.752930,10535.492188,-398.260742
5,10695.575195,10550.440430,-145.134766
6,10927.913086,18051.320312,7123.407227
7,19144.492188,30817.625000,11673.132812
8,35510.820312,46194.015625,10683.195312
9,52272.968750,52726.746094,453.777344


In [68]:
#Calcular o percentual de rentabilidade de cada ordem enviada

df_resultado['Resultado_%'] = ((df_resultado['Preco_venda'] - df_resultado['Preco_compra'])/df_resultado['Preco_compra'])
df_resultado

,Preco_compra,Preco_venda,Resultado,Resultado_%
0,6437.319336,9212.283203,2774.963867,0.431074
1,9438.914062,9386.035156,-52.878906,-0.005602
2,9273.357422,9238.703125,-34.654297,-0.003737
3,9162.514648,11773.588867,2611.074219,0.284974
4,10933.752930,10535.492188,-398.260742,-0.036425
5,10695.575195,10550.440430,-145.134766,-0.013570
6,10927.913086,18051.320312,7123.407227,0.651854
7,19144.492188,30817.625000,11673.132812,0.609738
8,35510.820312,46194.015625,10683.195312,0.300843
9,52272.968750,52726.746094,453.777344,0.008681


In [69]:
#Calcular a evolução do patrimonio
#Aqui vamos criar uma variavel caixa_inicial e atribuir 500000 para ela e depois vamos utilizar a função cumsum para acumular a coluna 'Resultado'
#que é referente aos resultados nominais de cada trade
caixa_inicial = 500000

df_resultado['EvolucaoPatrimonio'] = df_resultado['Resultado'].cumsum() + caixa_inicial

In [70]:
df_resultado

,Preco_compra,Preco_venda,Resultado,Resultado_%,EvolucaoPatrimonio
0,6437.319336,9212.283203,2774.963867,0.431074,502774.963867
1,9438.914062,9386.035156,-52.878906,-0.005602,502722.084961
2,9273.357422,9238.703125,-34.654297,-0.003737,502687.430664
3,9162.514648,11773.588867,2611.074219,0.284974,505298.504883
4,10933.752930,10535.492188,-398.260742,-0.036425,504900.244141
5,10695.575195,10550.440430,-145.134766,-0.013570,504755.109375
6,10927.913086,18051.320312,7123.407227,0.651854,511878.516602
7,19144.492188,30817.625000,11673.132812,0.609738,523551.649414
8,35510.820312,46194.015625,10683.195312,0.300843,534234.844727
9,52272.968750,52726.746094,453.777344,0.008681,534688.622070


In [71]:
df_result_pos = df_resultado[df_resultado["Resultado"]>0]
df_result_neg = df_resultado[df_resultado["Resultado"]<0]

In [72]:
#Plotando a evolução do patrimonio e também os trade com resultado líquido positivo (em verde) e negativo (em vermelho)

fig = make_subplots(rows=3, cols=1)
fig.add_trace(go.Scatter(x=df_resultado.index, 
                         y=df_resultado['EvolucaoPatrimonio'],
                         name='Evolução Patrimonio'),row=1,col=1)

fig.add_trace(go.Bar(x=df_result_pos.index, 
                     y=df_result_pos['Resultado'], 
                     name='PnL positivo',
                     marker_color = "green"), row=2,col=1)

fig.add_trace(go.Bar(x=df_result_neg.index, 
                     y=df_result_neg['Resultado'], 
                     name='PnL negativo',
                     marker_color = "red"), row=2,col=1)

fig.update_yaxes(title_text="<b> Valor Caixa (R$)", row=1, col=1)
fig.update_yaxes(title_text="<b> PnL Ops (R$)", row=2, col=1)
fig.update_layout(title_text='Evolução do Patrimônio e PnL',width=1000 ,height=800)
fig.show()